In [18]:
# Following:  https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
import torch.nn as nn
import torch
from tqdm import tqdm
import itertools

In [2]:
from datasets import load_dataset

dataset = load_dataset("lmqg/qa_wiki_t5_large")

Found cached dataset qa_wiki_t5_large (C:/Users/willc/.cache/huggingface/datasets/lmqg___qa_wiki_t5_large/default/0.0.0/f3a1d4d9e366c8e5d66c9329ca2a32b4cb673782cda308c7ef928789d431cfcb)


  0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="qa_wiki_t5_large_tokenizer.json")
fast_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [27]:
fast_tokenizer("abcedefghijklmnopqrstuvwxyz", return_tensors='pt')


{'input_ids': tensor([[70, 71, 72, 74, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
         87, 88, 89, 90, 91, 92, 93, 94, 95]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

In [28]:
def convert_to_single_string(sample):
    return sample['title'] + "[SEP]" + sample['context'] + "[ANS]" +sample['answers']['text'][0]

In [38]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def initHidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)
vocab_size = 2048
n_hidden = 512
batch_size = 64
model = RNN(vocab_size, n_hidden, vocab_size, batch_size)

In [39]:
crit = nn.CrossEntropyLoss()

In [40]:
def grouper(n, iterable):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk
        

In [41]:
torch.autograd.set_detect_anomaly(True)
def train(dataset, model):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(10):
        for batch, sample in enumerate(grouper(batch_size, dataset['train'])):
            state_h = model.initHidden()
            x = [convert_to_single_string(x) for x in sample]
            x = fast_tokenizer(x, return_tensors='pt', padding=True)['input_ids']
            x = torch.nn.functional.one_hot(x, vocab_size)
            
            optimizer.zero_grad()
            loss = torch.tensor([0.0])
            
            for word in tqdm(range(x.size()[1]-1)):
                
                y_pred, state_h = model(x[:, word], state_h)
                loss += crit(y_pred, x[:, word+1].float())/x.size()[1]

            loss.backward()
            optimizer.step()

            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item() })
        break
train(dataset, model)

100%|████████████████████████████████████████████████████| 888/888 [00:10<00:00, 82.82it/s]


{'epoch': 0, 'batch': 0, 'loss': 7.607909202575684}


100%|████████████████████████████████████████████████████| 895/895 [00:10<00:00, 84.79it/s]


{'epoch': 0, 'batch': 1, 'loss': 7.607226848602295}


100%|████████████████████████████████████████████████████| 852/852 [00:11<00:00, 74.99it/s]


...

100%|████████████████████████████████████████████████████| 832/832 [00:10<00:00, 82.22it/s]


{'epoch': 0, 'batch': 44, 'loss': 6.054130554199219}


100%|████████████████████████████████████████████████████| 819/819 [00:10<00:00, 77.02it/s]


{'epoch': 0, 'batch': 45, 'loss': 6.464259147644043}


RuntimeError: Class values must be smaller than num_classes.